# Classifier NNs

## NNs summary
- NNs components
    - Dense
    - BatchNormalization
    - DropOut
    - SoftMax

- Optimizer
    - Adam

- Loss
    - BinaryCrossEntropy

- Metric
    - acc

<img src="figures/NNs_classifier.png" style="float: left; width: 550px;"/>  

In [1]:
import pickle
import pprint
from sklearn.model_selection import train_test_split
from os import path

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
test_cases = [
    'pglib_opf_case24_ieee_rts.pickle', 
    'pglib_opf_case73_ieee_rts.pickle',
    'pglib_opf_case162_ieee_dtc.pickle',
    'pglib_opf_case300_ieee.pickle',
]

In [3]:
# choose a dataset
case_idx = 0
file_dir = path.join('./datasets/', test_cases[case_idx])

In [4]:
infile = open(file_dir,'rb')
dataset = pickle.load(infile)
infile.close()

In [5]:
# train & test
x_train, x_test, y_train, y_test = train_test_split(dataset['x'],
                                                    dataset['y'],
                                                    test_size=0.2,
                                                    random_state=19)
# train & validation
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=19)

In [6]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')

In [7]:
print((x_train.shape, y_train.shape))
print((x_val.shape, y_val.shape))
print((x_test.shape, y_test.shape))

((32000, 24), (32000, 82))
((8000, 24), (8000, 82))
((10000, 24), (10000, 82))


In [8]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [9]:
inputs = keras.Input(shape=(x_train.shape[1], ), name='uncertainty_realization')

x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.BatchNormalization(name='bn_1')(x)
x = layers.Dropout(rate=0.2, name='dropout_1')(x)

x = layers.Dense(128, activation='relu', name='dense_2')(x)
x = layers.BatchNormalization(name='bn_2')(x)
x = layers.Dropout(rate=0.2, name='dropout_2')(x)

x = layers.Dense(64, activation='relu', name='dense_3')(x)
x = layers.BatchNormalization(name='bn_3')(x)
x = layers.Dropout(rate=0.2, name='dropout_3')(x
                                              )
outputs = layers.Dense(y_train.shape[1], name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="naive_model")

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # List of metrics to monitor
    metrics=['accuracy'])

In [11]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [12]:
model.summary()

Model: "naive_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
uncertainty_realization (Inp [(None, 24)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1600      
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 64)                256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               

In [13]:
print('> Fit model on training data')
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=8,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
    callbacks=[tensorboard_callback])

> Fit model on training data
Train on 32000 samples, validate on 8000 samples
Epoch 1/8
32000/32000 [==============================] - 3s 90us/sample - loss: 2.5886 - accuracy: 0.7282 - val_loss: 1.1903 - val_accuracy: 0.8771
Epoch 2/8
32000/32000 [==============================] - 2s 63us/sample - loss: 1.2609 - accuracy: 0.8693 - val_loss: 0.6834 - val_accuracy: 0.9320
Epoch 3/8
32000/32000 [==============================] - 2s 63us/sample - loss: 0.7545 - accuracy: 0.9198 - val_loss: 0.3598 - val_accuracy: 0.9632
Epoch 4/8
32000/32000 [==============================] - 2s 62us/sample - loss: 0.4086 - accuracy: 0.9524 - val_loss: 0.1401 - val_accuracy: 0.9854
Epoch 5/8
32000/32000 [==============================] - 2s 65us/sample - loss: 0.1961 - accuracy: 0.9749 - val_loss: 0.0588 - val_accuracy: 0.9939
Epoch 6/8
32000/32000 [==============================] - 2s 65us/sample - loss: 0.0868 - accuracy: 0.9877 - val_loss: 0.0175 - val_accuracy: 0.9980
Epoch 7/8
32000/32000 [===========

In [14]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 30431), started 0:07:45 ago. (Use '!kill 30431' to kill it.)

In [15]:
pp = pprint.PrettyPrinter(indent=4)
print('> History dict:')
pp.pprint(history.history)

> History dict:
{   'accuracy': [   0.7282413,
                    0.86927074,
                    0.9198393,
                    0.9524184,
                    0.9748765,
                    0.98768777,
                    0.9940983,
                    0.99708295],
    'loss': [   2.5886276386976244,
                1.260883708357811,
                0.7544930895864963,
                0.4086447649002075,
                0.19606898499280215,
                0.08683328592404724,
                0.03619099695654586,
                0.015843785821867643],
    'val_accuracy': [   0.87712306,
                        0.9319648,
                        0.9632088,
                        0.98541665,
                        0.99390876,
                        0.99795896,
                        0.9995835,
                        0.99994504],
    'val_loss': [   1.190252249956131,
                    0.6833692067861556,
                    0.35976633605360986,
                    0.14005395326